In [1]:
import torch 

%cd ../


/workspace/Walk_Video_PyTorch


In [2]:
from  project.models.pytorchvideo_models import WalkVideoClassificationLightningModule

/usr/lib/python3.9/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [7]:
classification_module = WalkVideoClassificationLightningModule(model_type='resnet')
